In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

parent_dir = os.path.abspath(os.path.join(os.getcwd().split('render')[0],'gaussian_data'))
sys.path.insert(0, parent_dir)

from gaussian_data.Camera import Camera
from gaussian_data.Frame import Frame
import gaussian_data.Plotters
import gaussian_data.Utils as Utils
from GaussianSplat import GaussianSplat
import numpy as np
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1483]

image_names,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_names)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [2]:
def build_rotation(r):
    norm = np.sqrt(r[:,0]*r[:,0] + r[:,1]*r[:,1] + r[:,2]*r[:,2] + r[:,3]*r[:,3])

    q = r / norm[:, None]

    R = np.zeros((q.shape[0], 3, 3))

    r = q[:, 0]
    x = q[:, 1]
    y = q[:, 2]
    z = q[:, 3]

    R[:, 0, 0] = 1 - 2 * (y*y + z*z)
    R[:, 0, 1] = 2 * (x*y - r*z)
    R[:, 0, 2] = 2 * (x*z + r*y)
    R[:, 1, 0] = 2 * (x*y + r*z)
    R[:, 1, 1] = 1 - 2 * (x*x + z*z)
    R[:, 1, 2] = 2 * (y*z - r*x)
    R[:, 2, 0] = 2 * (x*z - r*y)
    R[:, 2, 1] = 2 * (y*z + r*x)
    R[:, 2, 2] = 1 - 2 * (x*x + y*y)
    return R


def build_scaling_rotation(s, r):
    L = np.zeros((s.shape[0], 3, 3))
    R = build_rotation(r)

    L[:,0,0] = s[:,0]
    L[:,1,1] = s[:,1]
    L[:,2,2] = s[:,2]

    L = R @ L
    return L

In [3]:

def homogeneous(points):
    """
    homogeneous points
    :param points: [..., 3]
    """
    return np.column_stack((points, np.ones(points.shape[0])))

def homogeneous_vec(vec, vectoadd = [0,0]):
    """
    homogeneous points
    :param points: [..., 3]
    """
    return np.concatenate((vec,np.tile(vectoadd,(vec.shape[0],1,1))),axis = 1)


In [57]:
temp_point = np.tile([1,1,-1],(T.shape[0],1))
distance = np.sum(temp_point*T[..., 3] * T[..., 3],1)[:,np.newaxis]
f = (1 / distance) * temp_point
f
point_image= np.column_stack((np.sum(f * T[..., 0] * T[...,3],1),np.sum(f * T[..., 1] * T[...,3],1),np.sum(f * T[..., 2] * T[...,3],1)))

axes_dist = np.column_stack((np.sum(f * T[..., 0] * T[...,0],1),np.sum(f * T[..., 1] * T[...,1],1),np.sum(f * T[..., 2] * T[...,2],1)))

half_extend = point_image * point_image - axes_dist

radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
center = point_image

AttributeError: 'numpy.ndarray' object has no attribute 'clamp'

In [64]:
half_extend

array([[-5.37883833e+19, -4.71569023e+19,  1.54046187e+16],
       [-1.20798088e+20, -3.16595370e+19, -3.85186388e+16],
       [-3.16464719e+18, -1.01904949e+20, -9.59147065e+16],
       ...,
       [-1.12801380e+20, -6.18272034e+19,  1.68840371e+16],
       [-1.00914136e+20, -3.10989262e+19, -2.57761141e+16],
       [-3.01647658e+19, -4.66180479e+19,  2.47638117e+16]])

In [55]:

# !! need to check that the order of axes ar ok for M !!
M = np.concatenate((homogeneous_vec(uv_view[:,:,:2]),homogeneous(p_view)[:,:,np.newaxis]),axis = 2)
M = np.moveaxis(M,[0,1,2],[0,2,1])
T = M @ projmat 

M

array([[[-5.38702526e+00, -8.14216646e+00, -6.34614017e-01,
          0.00000000e+00],
        [-8.61722142e-01, -1.28756271e-01,  9.82401441e+00,
          0.00000000e+00],
        [ 1.03553829e-02, -8.20026468e-01, -5.68537208e-01,
          1.00000000e+00]],

       [[-4.32023776e+00, -5.86982110e+00,  6.55004976e+00,
          0.00000000e+00],
        [ 6.89402000e+00,  3.23047517e+00,  6.91796999e+00,
          0.00000000e+00],
        [ 1.05706991e-02, -8.20053559e-01, -5.68501823e-01,
          1.00000000e+00]],

       [[-1.13130203e+00, -9.54128599e+00,  6.74520474e-03,
          0.00000000e+00],
        [ 5.67258216e+00, -5.95901392e-01, -7.87547101e+00,
          0.00000000e+00],
        [ 1.03501362e-02, -8.19985158e-01, -5.68528457e-01,
          1.00000000e+00]],

       ...,

       [[-2.06944815e-01, -6.54774534e+00,  6.81806175e+00,
          0.00000000e+00],
        [ 7.56687569e+00,  4.93636988e+00,  4.38286178e+00,
          0.00000000e+00],
        [ 1.08265215e-02

In [ ]:
   half_extend = point_image * point_image - torch.cat(
        [(f * T[..., 0] * T[...,0]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,1]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,2]).sum(dim=-1, keepdims=True)], dim=-1)

    radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
    center = point_image

array([[[  3.71500545,   8.17937147,   0.        ],
        [ -7.88096931,   4.97041867,   0.        ],
        [ -4.64015773,  -1.8933203 ,   0.        ]],

       [[ -1.00578775,   4.31913455,   0.        ],
        [-10.12343998,   1.12685217,   0.        ],
        [ -1.84639619,  -8.53107947,   0.        ]],

       [[ -8.98620414,  -3.6715601 ,   0.        ],
        [ -3.73751797,   8.83552612,   0.        ],
        [ -0.28818051,  -0.1025584 ,   0.        ]],

       ...,

       [[  1.03873477,   0.74379399,   0.        ],
        [ -9.81653402,   2.05385634,   0.        ],
        [ -2.15756664,  -8.98658035,   0.        ]],

       [[  0.83058196,   0.97693082,   0.        ],
        [ -7.78085833,  -5.65837812,   0.        ],
        [  5.76457079,  -7.77828194,   0.        ]],

       [[  1.56727529,   4.67732092,   0.        ],
        [ -7.79927796,   4.03020178,   0.        ],
        [ -5.95991502,  -4.04402014,   0.        ]]])

In [52]:
vertices['scale_0']

memmap([ -9.871272 , -10.3394785,  -9.7367325, ..., -10.104376 ,
         -9.7191515,  -9.940104 ], dtype=float32)

In [54]:
import numpy as np
input_file = "G:/My Drive/Research/gs_data/mov19_2022_03_03/2dgs_output/84444b22-6/point_cloud/iteration_1000/point_cloud.ply"
vertices = PlyData.read(input_file)["vertex"]




means3d = np.column_stack((vertices['x'],vertices['y'],vertices['z']))
viewmat = frames['P1483CAM1.jpg'].world_to_cam



r = np.column_stack([(vertices[f'rot_{idx}']) for idx in range(4)])
s = np.column_stack(([(vertices[f'scale_{idx}']) for idx in range(2)]))
s = np.column_stack((s,vertices['scale_0']))
rotations = build_scaling_rotation(s,r) # sutu, svtv (scale * axis direction in gaussian FoR)
projmat = frames['P1483CAM1.jpg'].projection


# 1. Viewing transform
# Eq.4 and Eq.5
p_view = (means3d @ viewmat[:3,:3]) + viewmat[-1:,:3] # rotate the gaussian mean to camera FoR
uv_view = (rotations @ viewmat[:3,:3]) # rotate to camera FoR

# M is H matrix that representes the transformation from tangent plane to camera. 
# its the scaled axes concatenated to the gaussian mean location - represented in homogeneous coordinates

# !! need to check that the order of axes ar ok for M !!
M = np.concatenate((homogeneous_vec(uv_view[:,:,:2]),homogeneous(p_view)[:,:,np.newaxis]),axis = 2)
M = np.moveaxis(M,[0,1,2],[0,2,1])
T = M @ projmat # T stands for (WH)^T in Eq.9 - projmat transforms from camera to NDC (screen coordinates)
# T is the transformation of every gaussian from tangent plane to NDC, its homogebnus coordinates. with the rotation matrix 
# representing the axes and the translation vector representing the location of the center of each gaussian. 
# We notice that projmat is a prespective projection matrix. 

# Next, We calculate the radius of the gaussian. We normalize by w to get homogeneus coordinates. In addition we flip Z axis (not sure why) 
# we calculate the distance from the camera to the gaussian mean (this is w, the last row of a homogenues coordinate, deviding by it will give perspective view)
# Notice that the rotation is scaled (in build_scaling_rotation) and is not normalized.

# point_image - the projectes mean of the gaussian (with flipped z)
# half_extend - used to calculate the radius of the gaussian, we take 3 sigma. because the ratation is scaled 
# we calculate the distance for each axis and can get the 3 sigma by multiplying each distance. (we also devide by w to get the prespective view)
temp_point = np.tile([1,1,-1],(T.shape[0],1))
distance  = np.sum(temp_point*T[..., 3] * T[..., 3],1)

distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
point_image = np.column_stack((np.sum(f * T[..., 0] * T[...,3],1),np.sum(f * T[..., 1] * T[...,3],1),np.sum(f * T[..., 2] * T[...,3],1)))

    # half_extend = point_image * point_image - torch.cat(
    #     [(f * T[..., 0] * T[...,0]).sum(dim=-1, keepdims=True),
    #     (f * T[..., 1] * T[...,1]).sum(dim=-1, keepdims=True),
    #     (f * T[..., 2] * T[...,2]).sum(dim=-1, keepdims=True)], dim=-1)

    # radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
    # center = point_image


TypeError: _sum() got an unexpected keyword argument 'dim'

In [9]:
T

array([[[ 7.11927951e+01,  6.80425005e+01, -8.78383233e-01,
          5.85476829e-10],
        [ 5.41476485e+01, -2.07705327e+01, -4.32812152e-01,
         -2.79772851e-09],
        [-1.37426825e-01,  6.83741628e+00,  7.21048663e-01,
          5.68537208e-10]],

       [[ 5.66293362e+01,  4.90257066e+01,  6.13121115e+00,
         -6.57562305e-09],
        [ 9.28555713e+00, -3.87876307e+01,  1.24622718e+00,
         -3.26116321e-09],
        [-1.40284297e-01,  6.83764217e+00,  7.21218439e-01,
          5.68501823e-10]],

       [[ 1.49900785e+01,  7.95549692e+01,  2.59406613e+00,
          4.95167258e-11],
        [-7.19222846e+01,  5.06695313e+00,  3.52654935e+00,
         -1.69604978e-10],
        [-1.37357195e-01,  6.83707184e+00,  7.21040082e-01,
          5.68528457e-10]],

       ...,

       [[ 1.97777543e+00,  5.45550951e+01,  9.00002903e+00,
         -6.82395961e-09],
        [ 9.48420363e+00, -3.53985041e+01,  1.65644059e+00,
         -3.53950828e-09],
        [-1.43679329e-01

In [ ]:
M.shape

(6079, 4, 4)

In [11]:
# Surface splatting (2D Gaussian Splatting)
def setup(means3D, scales, quats, opacities, colors, viewmat, projmat):
    rotations = build_scaling_rotation(scales, quats).permute(0,2,1)

    # 1. Viewing transform
    # Eq.4 and Eq.5
    p_view = (means3D @ viewmat[:3,:3]) + viewmat[-1:,:3]
    uv_view = (rotations @ viewmat[:3,:3])
    M = torch.cat([homogeneous_vec(uv_view[:,:2,:]), homogeneous(p_view.unsqueeze(1))], dim=1)

    T = M @ projmat # T stands for (WH)^T in Eq.9
    # 2. Compute AABB
    # Homogneous plane is very useful for both ray-splat intersection and bounding box computation
    # we know how to compute u,v given x,y homogeneous plane already; computing AABB is done by a reverse process.
    # i.e compute the x, y s.t. \|hu^4\| = 1 and \|h_v^4\|=1 (distance of gaussian center to plane in the uv space)
    temp_point = torch.tensor([[1.,1., -1.]]).cuda()
    distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
    f = (1 / distance) * temp_point
    point_image = torch.cat(
        [(f * T[..., 0] * T[...,3]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,3]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,3]).sum(dim=-1, keepdims=True)], dim=-1)

    half_extend = point_image * point_image - torch.cat(
        [(f * T[..., 0] * T[...,0]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,1]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,2]).sum(dim=-1, keepdims=True)], dim=-1)

    radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
    center = point_image

    # 3. Perform Sorting
    depth = p_view[..., 2] # depth is used only for sorting
    index = depth.sort()[1]
    T = T[index]
    colors = colors[index]
    center = center[index]
    depth = depth[index]
    radii = radii[index]
    opacities = opacities[index]
    return T, colors, opacities, center, depth, radii


## 2DGS - Math explained
### Modeling: 
The 2D Gaussian is defined in terms of its local coordinate system, where the $ X$ and $ Y$ axes are scaled according to the Gaussian's shape. The Gaussian is represented in the **world coordinates**, with the **axes** of the Gaussian in world space defined by the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $, and its **scaling factors** by $ s_u $ and $ s_v $. These tangential vectors define the directions of the local coordinate axes in the tangent plane (the object FoR).

The **object plane**, which is the tangent plane to the Gaussian in world space, can be described using the plane equation:
$$
P(u, v) = p_k + s_u \mathbf{t_u} u + s_v \mathbf{t_v} v
$$
where $ p_k $ is the center of the Gaussian in world coordinates, and $ u $ and $ v $ represent local coordinates on the tangent plane. This equation expresses the position of any point in the tangent plane in terms of the world coordinates, as modified by the scaling along the tangential axes $ \mathbf{t_u} $ and $ \mathbf{t_v} $.

To transform the local coordinates in the tangent plane (object frame of reference) to world coordinates, we define the following transformation matrix $ H $, which encodes the **scaling**, **rotation**, and **translation** from the object space to the world space:
$$
H = \begin{pmatrix}
s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 & p_k \\
0 & 0 & 0 & 1
\end{pmatrix}
= \begin{pmatrix}
R & p_k \\
0 & 1
\end{pmatrix}
$$
where:
- $ \mathbf{t_u} = \begin{pmatrix} t_{u_x} \\ t_{u_y} \\ t_{u_z} \end{pmatrix} $ and $ \mathbf{t_v} = \begin{pmatrix} t_{v_x} \\ t_{v_y} \\ t_{v_z} \end{pmatrix} $ are the 3D tangential vectors (defining the axes of the tangent plane in world coordinates),
- $ p_k = \begin{pmatrix} p_{k_x} \\ p_{k_y} \\ p_{k_z} \end{pmatrix} $ is the 3D position of the Gaussian center in world coordinates, and
- $ R $ is the 3x3 rotation matrix that describes the orientation of the tangent plane in world space.

The matrix $ H $ can be interpreted in two parts:
- The first part, $ \begin{pmatrix} s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 \end{pmatrix} $, represents the scaling and rotation of the axes in the world coordinates. It scales the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $ by $ s_u $ and $ s_v $, respectively, and applies any rotational transformation.
- The second part, $ p_k $, represents the **translation** of the Gaussian's center in world coordinates, which shifts the origin of the local tangent plane to the desired world position.

The matrix can also be interpreted in terms of the rotation matrix $ R $ (which describes the orientation of the tangent plane in world space) and the translation vector $ p_k $. The transformation from local coordinates $ (u, v) $ to world coordinates $ (x, y, z) $ is thus achieved through the multiplication of $ H $ by the local coordinate vector:
$$
\begin{pmatrix}
x \\
y \\
z \\
1
\end{pmatrix}
= H \begin{pmatrix}
u \\
v \\
1 \\
1
\end{pmatrix}
$$

This transformation allows us to map a point in the tangent plane (object space) to the corresponding point in the world space, accounting for both the Gaussian's shape (through the scaling $ s_u, s_v $) and its orientation (through the rotation $ R $).

In summary, the matrix $ H $ serves as a **homogeneous transformation** that encapsulates both the geometric properties (scaling and rotation) and the positioning (translation) of the 2D Gaussian in world space.

for every u,v on the tangent plane we can calculate the gaussian power: 
$$ G(u) = \exp\left( -\frac{u^2 + v^2}{2} \right) \tag{6} $$

* $ \mathbf{t_u} $ , $ \mathbf{t_v} , $ $\mathbf{s_u} $ and $\mathbf{s_v}$ are learnable parameters.
* each gaussian is defined by the opacity $\alpha$ and a view dependent color.

### Splatting


The goal of this process is to find the intersection between a ray, originating from a pixel in the image, and the tangent plane of a 2D Gaussian, and then evaluate the power of the Gaussian at the intersection point. Below are the detailed steps.

Step 1: Defining the Image Ray 

We begin by defining the ray from a pixel in the image using two orthogonal planes. The pixel location $(x, y) $ in the image space can be used to define two planes:


* The x-plane (yz): a plane defined by a normal vector $ \mathbf{n}_x = (-1, 0, 0) $ and an offset $ x $. The 4D homogeneous form is $ h_x = (-1, 0, 0, x) $.
* The y-plane (xz): a plane defined by a normal vector $ \mathbf{n}_y = (0, -1, 0) $ and an offset $ y $. The 4D homogeneous form is $ h_y = (0, -1, 0, y) $.


These planes intersect at a ray in 3D space that is represented as the line of intersection between the two planes.

Step 2: Transforming the Planes to the Tangent Plane

Next, we transform the planes from the image space into the local coordinates of the 2D Gaussian primitive (the tangent plane). Using the transformation matrix $ M = (WH)^{-1} $, we apply the inverse transpose to the planes (W is the prespective projection matrix (transformation from camera to screen space), H is a transformation from tangant plane to camera space):

multiplying W (camera to screen) by H (tangent plane to camera) will transform a coordinate from tangent plane to camera to screen. Transposing it will map from screen space to tangent plane 

$$
h_u = (WH)^\top h_x \\
h_v = (WH)^\top h_y
$$
* those are the planes rotated from image space to tangent space

Step 3: Solving for the Intersection

The next step is to solve for the intersection point $(u, v) $ of the ray with the transformed tangent plane. We do this by solving the following system of equations:

$$
h_u \cdot (u, v, 1, 1)^\top = 0 \\
h_v \cdot (u, v, 1, 1)^\top = 0
$$

Expanding these equations gives us:


$$
    h_1^u u + h_2^u v + h_3^u + h_4^u = 0 \\
    h_1^v u + h_2^v v + h_3^v + h_4^v = 0
$$


The solution for $ u $ and $ v $ is obtained by solving this system of equations. This leads to the following expressions for the coordinates $ u(x) $ and $ v(x) $:

$$
u = \frac{h_2^u h_4^v - h_4^u h_2^v}{h_1^u h_2^v - h_2^u h_1^v} \\
v = \frac{h_4^u h_1^v - h_1^u h_4^v}{h_1^u h_2^v - h_2^u h_1^v}
$$

Where $ h_i^u $ and $ h_i^v $ are the homogeneous parameters of the transformed planes.

Step 4: Evaluating the Gaussian at the Intersection

Once we have the coordinates $ (u, v) $, we can evaluate the 2D Gaussian function at the intersection point. The Gaussian function is typically of the form:

$$
G(u, v) = \exp\left( -\frac{u^2 + v^2}{2} \right)
$$

This represents the Gaussian value at the point $ (u, v) $ on the tangent plane.


Once the values of $ u(x) $ and $ v(x) $ are found, we can compute the depth of the intersection point using the following equation:
$$
x = (x_z, y_z, z, z)^\top = W P(u, v) = W H (u, v, 1, 1)^\top
$$
Here, $ W $ is the camera projection matrix, and $ H $ is the transformation matrix from the tangent plane to world coordinates. The last component of the resulting vector $ x = (xz, yz, z, z)^\top $ gives the depth $ z $ of the intersection point (the third element, need to make sure). 



#### Screen space
this transformation returns a 3d coordinate in screen space. 
, where rendering occures. the projection matrix transformes x,y,z to the screen space
* x axis spans from 0 to -1 (width)
* y axis spans from 0 to -1 (hight)
* Z_{NDC} represents the depth (need to check that)
from NDC we can map to pixels

#### Transforming planes
* A point in 3D homogeneous coordinates is represented as $ (x,y,z,w)
* a plane consist of both a normal and an offset - in 3D homogeneous coordinates it is represented as $(a,b,c,d)
where $(a,b,c) is the planes normal vector and $d$ is its offset. 

The plane equation : $ ax + by + cz + dw = 0 $
which means that that dot product of the plane parameters $ (a,b,c,d)$ and a point $(x,y,z,w)$ is zero for a point lying on the plane. 

** Transforming Point on a Plane **
we Define matrix $M$ - a transformation matrix. 
tansforming a point on the plain using $M$: $p'=M\cdot p$
after transforming it, the plane equation should still hold: $h'p' = 0$ were $h', p'$ are the transformed plane and point parameters.

Because $p' = M\cdot p$ we get that $h'(M\cdot p) = 0 $ and since $ h' \cdot(M\cdotp p)=M^T \cdot h' \cdot p $ we can write 
$$
M^T \cdot h' \cdot p = h\cdot p \\
M^T \cdot h' = h
$$

and from that we show that the plane $h'$ remains valid after transformation. 
